In [2]:
#Pyspark not needed for this script, jsut testing
#Aban
import pyspark
from pyspark.sql import SparkSession

from pyspark.ml.feature import MinHashLSH

In [3]:
import sys
import json
import time
import os

In [5]:
import pandas as pd

In [6]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
import scipy.sparse

In [7]:
spark = SparkSession \
    .builder \
    .master("local[4]") \
    .appName("SparkTest") \
    .getOrCreate()   

In [8]:
data_location = "/home/ubutubiggerbetter/shared/data/spotify_data/data/formatted_data"
NUM_PLAYLISTS = 1000000
NUM_ARTISTS = 295860 #<- from stats.txt

In [9]:
#Helpful commands:
#type() tells type
#sys.getsizeof(obj) tell size in bytes of obj
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1646423603158'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.startTime', '1646423599280'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '42961'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.warehouse.dir',
  'file:/home/taylor/Desktop/CMPE256/proj/CMPE256_Project/spark-warehouse')]

In [11]:
filenames = os.listdir(data_location)
test_file = filenames[1]
sparse_mat = scipy.sparse.load_npz(os.sep.join((data_location, test_file)))
sparse_mat_red = sparse_mat.astype(np.uint16)
raw_data = sparse_mat_red.toarray()
rdd = spark.sparkContext.parallelize([raw_data])
print(rdd)

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:274


In [22]:
#This general approach fails:
#pd_sparse_mat = pd.DataFrame.sparse.from_spmatrix(sparse_mat)
#spark.createDataFrame(pd_sparse_mat)
test_list = []
if not test_list:
    print('test')
else:
    print('test2')

test


In [10]:
df = rdd.toDF()

22/03/02 22:27:50 WARN TaskSetManager: Stage 0 contains a task of very large size (587795 KiB). The maximum recommended task size is 1000 KiB.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
mh = MinHashLSH()

In [ ]:
test_index = 1
filenames = os.listdir(data_location)
#filenames = filenames[0:test_index+1]
playlist_index = 0
artist_dict = dict()
artist_id_array = []
total_artist_count = 0
charmat = csc_matrix((NUM_ARTISTS,1000))
file_index = 0
for filename in sorted(filenames):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        f = open(fullpath)
        js = f.read()
        f.close()
        mpd_slice = json.loads(js)
        start_time = time.time()
        print('Filename: ', filename)
        charmat = csc_matrix((NUM_ARTISTS,1000))
        playlist_index = 0
        for playlist in mpd_slice["playlists"]:
            print('p',playlist_index, sep='', end=', ')
            #Per each track in given playlist:
            
            for track in playlist['tracks']:                
                artist_name = track['artist_name']
                #if arist is in charmat increment that location, 
                #else save off artist name to create new addition to 
                #charmat
                if artist_name in artist_dict:
                    #Already identified artist
                    artist_index = artist_dict[artist_name]
                    charmat[artist_index, playlist_index] = charmat[artist_index, playlist_index] + 1
                    
                else:
                    #Brand new artist:
                    #add to dictionary w/index
                    artist_dict[artist_name] = total_artist_count
                    #add to matrix
                    charmat[total_artist_count, playlist_index] = 1                    
                    #increment count
                    total_artist_count = total_artist_count + 1
                    if total_artist_count >= NUM_ARTISTS:
                        print('Total artist count is larger than spotify info. Will cause error')                
                

            #increment playlist id:
            playlist_index = playlist_index + 1
        print('Time elapsed for file', filename, '= ', time.time() - start_time)
        fname = os.sep.join(('formatted_data', 'charmat_pt_' + str(file_index)))
        scipy.sparse.save_npz(os.sep.join((data_location, fname)), charmat)
        file_index = file_index + 1
            